In [77]:
import numpy as np  
import os 
from matplotlib import pyplot as plt 
import math

currentDir = os.getcwd()
firstPath = os.path.join(currentDir,"dataFirstMethod.txt")
secondPath = os.path.join(currentDir,'dataSecondMethod.txt')

In [78]:
def convertNumbersRA(inputString):
    prescalers = [15,15/60,15/3600] # Make them degrees, arcminutes,arcseconds
    finalDegree = 0
    for time, number in zip(inputString,prescalers):
        finalDegree += float(time)*number
    return finalDegree

def convertNumbersDELTA(inputString):
    prescalers = [1,1/60,1/3600] # Make them degrees, arcminutes,arcseconds
    finalDegree = 0
    for time, number in zip(inputString,prescalers):
        finalDegree += float(time)*number
    return finalDegree

In [79]:
firstMethodData = {"NAME":[],"RA":[],"DELTA":[]}
secondMethodData = {
    'PARALLAX' : [],
    'PARALLAX_ERROR' : [],
    'VELOCITY' : [],
    'VELOCITY_ERROR' : [],
    'MU_A_COS(DELTA)' : [],
    'MU_A_COS(DELTA)_ERROR' : [],
    'MU_DELTA' : [],
    'MU_DELTA_ERROR' : []
}
keysArray = ['PARALLAX',
    'PARALLAX_ERROR',
    'VELOCITY',
    'VELOCITY_ERROR', 
    'MU_A_COS(DELTA)',
    'MU_A_COS(DELTA)_ERROR',
    'MU_DELTA' ,
    'MU_DELTA_ERROR']

with open(firstPath, 'r') as file:
    line = file.readline()
    while line:
        currentLine = line.strip("\n").split(" ")
        RAString, DELTAString = currentLine[1:4] , currentLine[4:7] #Make all the angles into degrees
        firstMethodData["NAME"].append(currentLine[0])
        firstMethodData["RA"].append(convertNumbersRA(RAString))
        firstMethodData["DELTA"].append(convertNumbersDELTA(DELTAString))
        line = file.readline()
        
with open(secondPath, 'r') as file:
    for line in file:
        currentLine = line.strip("\n").split(" ")[1::] #Clear the line by ignoring the first elements that correspond to the name
        #print(currentLine)
        for index, value in enumerate(currentLine):
            trueValue,trueError = value.split('±')
            keyIdValue,keyIdError = keysArray[index*2],keysArray[index*2+1]
            secondMethodData[keyIdValue].append(float(trueValue))
            secondMethodData[keyIdError].append(float(trueError))
            
for keyToPop in keysArray[2::]:
    firstMethodData[keyToPop] = secondMethodData.pop(keyToPop)
secondMethodData['NAME'] = firstMethodData['NAME']


In [80]:
#The formula for finding the difference between a star (alpha,delta) and the convergent point(alpha_conv,delta_conv) is:
#cos(theta) = sin(delta)*sin(delta_c)+cos(delta)*cos(delta_c)*cos(alpha-alpha_conv)
def findAngleTheta(alpha,delta): #Takes as arguments the RA and DELTA values of the dict row and returns the assigned THETA angle in degrees
    alpha_conv,delta_conv = 96.6, 5.8
    degreeToRadians = math.pi/180
    part1 = math.sin(delta*degreeToRadians)*math.sin(delta_conv*degreeToRadians)
    part2 = math.cos(delta*degreeToRadians)*math.cos(delta_conv*degreeToRadians)*math.cos((alpha-alpha_conv)*degreeToRadians)
    return math.acos(part1 + part2)/degreeToRadians

def addThetaAngleColumn(dictionary): #Takes the dictionary of data and returns the array of all the THETA angles in degrees
    if 'THETA' in dictionary:
        return dictionary['THETA']
    deltaArray,alphaArray = dictionary['DELTA'],dictionary['RA']
    thetaArray = []
    for alpha,delta in zip(alphaArray,deltaArray):
        thetaArray.append(findAngleTheta(alpha,delta))
    return thetaArray

def addTheMuColumn(dictionary):
    if 'MU' in dictionary:
        return dictionary['MU']
    MU_DELTA, MU_A_COS = dictionary['MU_DELTA'], dictionary['MU_A_COS(DELTA)']
    MuArray = []
    for muDelta,muA in zip(MU_DELTA,MU_A_COS):
        MuArray.append(math.sqrt(muDelta**2 + muA**2))
    return MuArray   

def addTheDistanceColumn(dictionary):
    if 'DISTANCE' in dictionary:
        return dictionary['DISTANCE']
    thetaArray,muArray,velocityArray = dictionary['THETA'], dictionary['MU'], dictionary['VELOCITY']
    degreeToRadians = math.pi/180
    distanceArray = []
    for theta,mu,velocity in zip(thetaArray,muArray,velocityArray):
        distanceArray.append(1000*velocity*math.tan(theta*degreeToRadians)/(4.74*mu))
    return distanceArray

def addTheErrorColumn(dictionary):
    if 'ERROR_DISTANCE' in dictionary:
        return dictionary['ERROR_DISTANCE']
    xArray,errorxArray,yArray,erroryArray,muArray = dictionary['MU_A_COS(DELTA)'],dictionary['MU_A_COS(DELTA)_ERROR'],dictionary['MU_DELTA'],dictionary['MU_DELTA_ERROR'],dictionary['MU']
    muErrorArray,distanceErrorArray = [], []
    for x,errx,y,erry,mu in zip(xArray,errorxArray,yArray,erroryArray,muArray):
        muErrorArray.append(math.sqrt((x*errx)**2+(y*erry)**2)/mu)
    velArray,distanceArray,velocityErrorArray = dictionary['VELOCITY'], dictionary['DISTANCE'],dictionary['VELOCITY_ERROR']
    for vel,velerr,mu,muerr,r in zip(velArray,velocityErrorArray,muArray,muErrorArray,distanceArray):
        distanceErrorArray.append(r*math.sqrt((velerr/vel)**2+(muerr/mu)**2))
    return distanceErrorArray
    

In [81]:
firstMethodData.update({'THETA': addThetaAngleColumn(firstMethodData)})
firstMethodData.update({'MU': addTheMuColumn(firstMethodData)})
firstMethodData.update({'DISTANCE': addTheDistanceColumn(firstMethodData)})
firstMethodData.update({'DISTANCE_ERROR':addTheErrorColumn(firstMethodData)})

In [82]:
def addSecondMethodDistance(dictionary): 
    if 'DISTANCE' in dictionary:
        return dictionary['DISTANCE']
    distanceArray,parallaxArray = [], dictionary['PARALLAX']
    for par in parallaxArray:
        distanceArray.append(1000/par)
    return distanceArray  

def addSecondMethodDistanceError(dictionary):
    if 'DISTANCE_ERROR' in dictionary:
        return dictionary['DISTANCE_ERROR']
    parallaxArray, parallaxErrorArray,distanceArray = dictionary['PARALLAX'],dictionary['PARALLAX_ERROR'],dictionary['DISTANCE']
    distanceErrorArray = []
    for par,parerror,dis in zip(parallaxArray, parallaxErrorArray,distanceArray):
        distanceErrorArray.append(par*parerror/dis)
    return distanceErrorArray
    

In [83]:
secondMethodData.update({'DISTANCE': addSecondMethodDistance(secondMethodData)})
secondMethodData.update({'DISTANCE_ERROR': addSecondMethodDistanceError(secondMethodData)})

In [84]:
def clearData(dictionaryFirstMethod,dictionarySecondMethod):
    thresHold,center = 10 ,46.34
    totalData = {
        'NAME':[],
        'DISTANCE_FIRST':[],
        'DISTANCE_FIRST_ERROR':[],
        'DISTANCE_SECOND':[],
        'DISTANCE_SECOND_ERROR' : []
        }
    distArray = dictionaryFirstMethod['DISTANCE']
    for index,starDist in enumerate(distArray):
        if (abs(starDist - center) < thresHold):
            totalData['NAME'].append(dictionaryFirstMethod['NAME'][index])
            totalData['DISTANCE_FIRST'].append(dictionaryFirstMethod['DISTANCE'][index])
            totalData['DISTANCE_FIRST_ERROR'].append(dictionaryFirstMethod['DISTANCE_ERROR'][index])
            totalData['DISTANCE_SECOND'].append(dictionarySecondMethod['DISTANCE'][index])
            totalData['DISTANCE_SECOND_ERROR'].append(dictionarySecondMethod['DISTANCE_ERROR'][index])
    return totalData


In [85]:
totalData = clearData(firstMethodData,secondMethodData)

In [89]:
for name,val1,val2 in zip(totalData['NAME'],totalData['DISTANCE_FIRST'],totalData['DISTANCE_SECOND']):
    print(name,val1,val2)

14838 39.107610614413005 51.440329218106996
18170 40.9020048222674 41.42502071251035
18735 38.854267136834224 45.47521600727604
20205 44.94890902880092 47.23665564478035
20261 43.507543790591804 47.16981132075472
20400 42.412913068256664 45.72473708276177
20455 47.51629744857027 46.97040864255519
20542 45.06187039141562 44.72271914132379
20635 47.21723002046411 47.01457451810061
20711 42.26942382824494 47.460844803037496
20713 43.373743222239035 47.93863854266539
20842 47.55534036219393 47.961630695443645
20885 47.37529356118704 48.402710551790896
20889 45.12621729544459 47.528517110266165
20894 43.341932902733035 45.682960255824575
20901 45.48097529589755 49.18839153959666
21029 46.66856162612917 44.36557231588288
21036 42.287867286550224 45.78754578754579
21039 45.14830119710423 44.34589800443459
21137 38.74319014775086 44.943820224719104
21152 39.78641571816683 43.2338953739732
21459 46.14636094537459 44.24778761061947
21589 47.795566368239356 45.892611289582376
21683 47.43024674097